In [1]:
import re
import csv
from collections import defaultdict

md5_regex = r'([a-fA-F\d]{32})' # This regular expression matches a string of 32 hexadecimal digit characters, which is the typical format for an MD5 hash.
sha1_regex = r'([a-fA-F\d]{40})' # This regular expression matches a string of 40 hexadecimal digit characters, which is the standard length for a SHA1 hash.
sha256_regex = r'([a-fA-F\d]{64})' # Similar to sha1 but with 64
ipv4_regex = r'^(25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[0-1]?[0-9][0-9]?)$'
ipv6_regex = r'((([0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4})|(([0-9a-fA-F]{1,4}:){1,7}:)|(([0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4})|(([0-9a-fA-F]{1,4}:){1,5}(:[0-9a-fA-F]{1,4}){1,2})|(([0-9a-fA-F]{1,4}:){1,4}(:[0-9a-fA-F]{1,4}){1,3})|(([0-9a-fA-F]{1,4}:){1,3}(:[0-9a-fA-F]{1,4}){1,4})|(([0-9a-fA-F]{1,4}:){1,2}(:[0-9a-fA-F]{1,4}){1,5})|([0-9a-fA-F]{1,4}:((:[0-9a-fA-F]{1,4}){1,6})|:((:[0-9a-fA-F]{1,4}){1,7}|:))|fe80:(:[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(ffff(:0{1,4}){0,1}:){0,1}((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])|([0-9a-fA-F]{1,4}:){1,4}:((25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(25[0-5]|(2[0-4]|1{0,1}[0-9]){0,1}[0-9]))'
domain_regex = r'\b(?:[a-z0-9|-]+\.)*[a-z0-9|-]+\.[a-z]+\b' # Domain names present in network traffic.
c2_regex = r'C2:\s*(.+)' # Command and Control (C2) information embedded in network protocols.
uri_regex = r'URI: (.+)' # URIs
user_agent_regex = r'User-Agent: (.+)' # user agent
smtp_mailer_regex = r'X-Mailer:\s*(.+)' # SMTP (Simple Mail Transfer Protocol) mailer values in email headers.
mac_address_regex = r'([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})' # Media Access Control (MAC) addresses used in network communication.
http_method_regex = r'(GET|POST|PUT|DELETE)' # HTTP request methods such as GET, POST, PUT, DELETE.
port_number_regex = r'(?<!\d)(\d{1,5})(?!\d)' # TCP and UDP port numbers used in network communication.
email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b' # Email addresses transmitted in network data.
cipher_suite_regex = r'(TLS_(?:DHE|RSA|ECDHE)_WITH_AES_\d{3}_GCM_SHA\d{2})' # Cipher suites used in SSL/TLS communication.
hostname_regex = r'([a-zA-Z0-9][a-zA-Z0-9-]{,61}[a-zA-Z0-9]\.)+[a-zA-Z]{2,}' # hostname_regex = r'([a-zA-Z0-9][a-zA-Z0-9-]{,61}[a-zA-Z0-9]\.)+[a-zA-Z]{2,}'
file_path_regex = r'https?://[^/]+(/[^/\s]+\.\w+)' # file path
mime_type_regex = r'(application|audio|image|message|model|multipart|text|video)/([a-zA-Z0-9.-]+)' # Multipurpose Internet Mail Extensions (MIME) types used in email and web protocols.
http_status_code_regex = r'\b\d{3}\b' # HTTP status codes indicating the result of an HTTP request.
asn_regex = r'\bAS\d+\b' # Border Gateway Protocol (BGP) Autonomous System Numbers (ASNs) used for routing.
subnet_mask_regex = r'(\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})/\d{1,2}\b' # Subnet masks used to divide an IP network into subnetworks.
snmp_community_regex = r'community\s+(\S+)' # Simple Network Management Protocol (SNMP) community strings used for authentication.
dns_record_regex = r'\b(?:A|AAAA|CNAME|MX|NS|PTR|SOA|SRV|TXT)\b' # DNS resource records containing information about a domain name.
tls_version_regex = r'(TLSv1\.2|TLSv1\.3)' # TLS version information used in network communication.
cookie_regex = r'Set-Cookie:\s*(.+)' # HTTP cookies used for session management in web protocols.
ssh_fingerprint_regex = r'\b([0-9a-f]{2}:){15}[0-9a-f]{2}\b' # SSH key fingerprints used for secure authentication.
snmp_oid_regex = r'\.(?:\d+\.)*\d+' # SNMP Object Identifiers (OIDs) used for uniquely identifying managed objects in a database.
protocol_name_regex = r'(HTTP|SSL|DNS|FTP|SMTP)' # Names of network protocols such as HTTP, SSL, DNS.
# timestamp_regex = r'\b\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\b' # Timestamps indicating the time of network events.
snmp_trap_regex = r'SNMP Trap: (.+)' # SNMP trap information used for network monitoring and management.
http_header_regex = r'(Accept|Content-Type|Cache-Control): (.+)' # Various HTTP header fields such as Accept, Content-Type, Cache-Control, etc.
interface_name_regex = r'(eth|wlan|enp)[0-9a-zA-Z]+' # Names of network interfaces such as eth0, wlan0, enp0s3, etc.
arp_entry_regex = r'([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2})\s+(\d{1,3}\.){3}\d{1,3}' # MAC to IP address mappings in ARP (Address Resolution Protocol) tables.
service_name_regex = r'(ssh|http|ftp|dns)'# Names of network services such as ssh, http, ftp, dns, etc.
snmp_mib_regex = r'1\.3\.6\.1\.\d+\.\d+' # SNMP Management Information Base (MIB) objects used for network management.
device_name_regex = r'(router|switch|firewall)' # Names of network devices such as router, switch, firewall
ssl_certificate_regex = r'Subject: CN=(.*?)(?:,|$)' # SSL/TLS certificate information including common name, organization, and validity period.
routing_protocol_regex = r'(OSPF|BGP|RIP)' # Information related to network routing protocols such as OSPF, BGP, RIP, etc.
packet_capture_timestamp_regex = r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}\.\d+' # Timestamps associated with network packet captures.
traffic_volume_regex = r'(\d+)(\sbytes|packets)' # Information about network traffic volume, such as bytes transferred, packets sent/received, etc.
qos_marking_regex = r'(EF|AF[1-4]|CS[1-7])' # Quality of Service (QoS) markings in network traffic for prioritization.
protocol_field_regex = r'(IP|TCP|UDP)\.(.+)'  # Fields within network protocol headers such as IP, TCP, UDP, etc.
virtualization_tech_regex = r'(VLAN|VXLAN)' # Information related to network virtualization technologies such as VLAN, VXLAN, etc.
security_protocol_regex = r'(IPSec|SSL/TLS|WPA2)' # Names of network security protocols such as IPSec, SSL/TLS, WPA2, etc.
tunneling_protocol_regex = r'(GRE|L2TP|PPTP)'  # Information related to network tunneling protocols such as GRE, L2TP, PPTP, etc.
auth_method_regex = r'(EAP|Kerberos|RADIUS)' # Methods of network authentication such as EAP, Kerberos, RADIUS, etc.
file_transfer_protocol_regex = r'(FTP|SFTP|SCP)' # Names of network file transfer protocols such as FTP, SFTP, SCP, etc.
transport_security_regex = r'(DTLS|TLS 1\.3)' # Information related to transport layer security protocols such as DTLS, TLS 1.3, etc.
nat_info_regex = r'(Source NAT|Destination NAT)' # Information related to Network Address Translation (NAT) such as source NAT, destination NAT, etc.
proxy_server_regex = r'(Forward Proxy|Reverse Proxy)' # Information related to network proxy servers such as forward proxy, reverse proxy, etc.

In [2]:
regex_list = [
    ("md5_regex", md5_regex),
    ("sha1_regex", sha1_regex),
    ("sha256_regex", sha256_regex),
    ("ipv4_regex", ipv4_regex),
    ("ipv6_regex", ipv6_regex),
    ("domain_regex", domain_regex),
    ("c2_regex", c2_regex),
    ("uri_regex", uri_regex),
    ("user_agent_regex", user_agent_regex),
    ("smtp_mailer_regex", smtp_mailer_regex),
    ("mac_address_regex", mac_address_regex),
    ("http_method_regex", http_method_regex),
    ("port_number_regex", port_number_regex),
    ("email_regex", email_regex),
    ("cipher_suite_regex", cipher_suite_regex),
    ("hostname_regex", hostname_regex),
    ("file_path_regex", file_path_regex),
    ("mime_type_regex", mime_type_regex),
    ("http_status_code_regex", http_status_code_regex),
    ("asn_regex", asn_regex),
    ("subnet_mask_regex", subnet_mask_regex),
    ("snmp_community_regex", snmp_community_regex),
    ("dns_record_regex", dns_record_regex),
    ("tls_version_regex", tls_version_regex),
    ("cookie_regex", cookie_regex),
    ("ssh_fingerprint_regex", ssh_fingerprint_regex),
    ("snmp_oid_regex", snmp_oid_regex),
    ("protocol_name_regex", protocol_name_regex),
    ("snmp_trap_regex", snmp_trap_regex),
    ("http_header_regex", http_header_regex),
    ("interface_name_regex", interface_name_regex),
    ("arp_entry_regex", arp_entry_regex),
    ("service_name_regex", service_name_regex),
    ("snmp_mib_regex", snmp_mib_regex),
    ("device_name_regex", device_name_regex),
    ("ssl_certificate_regex", ssl_certificate_regex),
    ("routing_protocol_regex", routing_protocol_regex),
    ("packet_capture_timestamp_regex", packet_capture_timestamp_regex),
    ("traffic_volume_regex", traffic_volume_regex),
    ("qos_marking_regex", qos_marking_regex),
    ("protocol_field_regex", protocol_field_regex),
    ("virtualization_tech_regex", virtualization_tech_regex),
    ("security_protocol_regex", security_protocol_regex),
    ("tunneling_protocol_regex", tunneling_protocol_regex),
    ("auth_method_regex", auth_method_regex),
    ("file_transfer_protocol_regex", file_transfer_protocol_regex),
    ("transport_security_regex", transport_security_regex),
    ("nat_info_regex", nat_info_regex),
    ("proxy_server_regex", proxy_server_regex)
]

In [5]:
# Create a dictionary to store rows for each regex pattern
matches_dict = {name: [] for name, _ in regex_list}

# Read the CSV content and match regular expressions
with open('allsources_3x200k.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    headers = csv_reader.fieldnames  # Get the headers from the original CSV
    
    for row in csv_reader:
        content = row['content']
        
        # Match regular expressions
        for name, regex in regex_list:
            if re.search(regex, content):
                matches_dict[name].append(row)

# Create a dictionary to store counts for each regex pattern
regex_counts = {key: len(value) for key, value in matches_dict.items()}

# Print the dictionary with regex counts
print(dict(regex_counts))

# Write the matches and counts to a new CSV file
with open('output.csv', 'w', newline='') as file:
    csv_writer = csv.writer(file)
    
    # Write headers
    csv_writer.writerow(headers + ['regex_name', 'count'])
    
    # Write rows, regex names, and their counts
    for key, rows in matches_dict.items():
        count = regex_counts[key]
        for row in rows:
            csv_writer.writerow([row[header] for header in headers] + [key, count])

{'md5_regex': 5, 'sha1_regex': 4, 'sha256_regex': 3, 'ipv4_regex': 0, 'ipv6_regex': 0, 'domain_regex': 147598, 'c2_regex': 0, 'uri_regex': 0, 'user_agent_regex': 0, 'smtp_mailer_regex': 0, 'mac_address_regex': 0, 'http_method_regex': 0, 'port_number_regex': 0, 'email_regex': 0, 'cipher_suite_regex': 0, 'hostname_regex': 136864, 'file_path_regex': 0, 'mime_type_regex': 0, 'http_status_code_regex': 0, 'asn_regex': 0, 'subnet_mask_regex': 0, 'snmp_community_regex': 37229, 'dns_record_regex': 0, 'tls_version_regex': 0, 'cookie_regex': 0, 'ssh_fingerprint_regex': 0, 'snmp_oid_regex': 0, 'protocol_name_regex': 0, 'snmp_trap_regex': 0, 'http_header_regex': 0, 'interface_name_regex': 88213, 'arp_entry_regex': 0, 'service_name_regex': 25225, 'snmp_mib_regex': 0, 'device_name_regex': 6332, 'ssl_certificate_regex': 0, 'routing_protocol_regex': 0, 'packet_capture_timestamp_regex': 0, 'traffic_volume_regex': 0, 'qos_marking_regex': 0, 'protocol_field_regex': 0, 'virtualization_tech_regex': 0, 'secu

In [6]:
import pandas as pd

def find_ids_and_frequencies_by_names(software_file, data_file, output_file):
    # Load the software Excel file and focus on the "name" column
    software_df = pd.read_excel(software_file, sheet_name='software')
    names = software_df['name'].tolist()

    # Load the data CSV file
    data_df = pd.read_csv(data_file)
    
    # Initialize lists to store results
    results = []

    # Iterate over each name and check if it is found in any content
    for name in names:
        matching_rows = data_df[data_df['content'].str.contains(str(name), na=False)]
        frequency = len(matching_rows)
        
        if not matching_rows.empty:
            for _, row in matching_rows.iterrows():
                result_row = row.to_dict()
                result_row['software_name'] = name
                result_row['frequency'] = frequency
                results.append(result_row)

    # Convert results to DataFrame and save to CSV
    if results:
        results_df = pd.DataFrame(results)
        results_df.to_csv(output_file, index=False)
    else:
        print("No matches found.")

# Usage
software_file = 'software.xlsx'  # Replace with your software Excel file path
data_file = 'allsources_3x200k.csv'  # Replace with your data CSV file path
output_file = 'output1.csv'  # Output CSV file path

find_ids_and_frequencies_by_names(software_file, data_file, output_file)

C:\Users\irdin\AppData\Local\Temp\ipykernel_33684\872268648.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(data_file)


In [4]:
import pandas as pd

def find_ids_and_frequencies_by_names(software_file, data_file):
    # Load the software Excel file and focus on the "name" column
    software_df = pd.read_excel(software_file, sheet_name='software')
    names = software_df['name'].tolist()

    # Load the data CSV file and focus on the "content" and "id" columns
    data_df = pd.read_csv(data_file)
    contents = data_df['content'].tolist()
    ids = data_df['id'].tolist()

    # Initialize the result dictionaries
    result_dict = {}
    frequency_dict = {}

    # Iterate over each name and check if it is found in any content
    for name in names:
        print(name)
        matching_ids = [ids[i] for i, content in enumerate(contents) if str(name) in str(content)]
        frequency = sum(str(name) in str(content) for content in contents)
        if matching_ids:
            result_dict[name] = (matching_ids, frequency)
        if frequency > 0:
            frequency_dict[name] = frequency

    return result_dict, frequency_dict

# Usage
software_file = 'software.xlsx'  # Replace with your software Excel file path
data_file = 'all_data.csv'  # Replace with your data Excel file path

result_dict, frequency_dict = find_ids_and_frequencies_by_names(software_file, data_file)
print("Result Dictionary:", result_dict)
print("Frequency Dictionary:", frequency_dict)

3PARA RAT
4H RAT
AADInternals
ABK
ADVSTORESHELL
ANDROMEDA
ASPXSpy
AcidRain
Action RAT
AdFind
Agent Tesla
Agent.btz
Akira
Amadey
Anchor
AppleJeus
AppleSeed
Aria-body
Arp
Astaroth
AsyncRAT
Attor
AuTo Stealer
AuditCred
AutoIt backdoor
Avaddon
Avenger
AvosLocker
Azorult
BACKSPACE
BADCALL
BADFLICK
BADHATCH
BADNEWS
BBK
BBSRAT
BISCUIT
BITSAdmin
BLACKCOFFEE
BLINDINGCAN
BLUELIGHT
BONDUPDATER
BOOSTWRITE
BOOTRASH
BS2005
BUBBLEWRAP
BUSHWALK
Babuk
BabyShark
BackConfig
Backdoor.Oldrea
Bad Rabbit
BadPatch
Bandook
Bankshot
Bazar
BendyBear
Bisonal
BitPaymer
Black Basta
BlackCat
BlackEnergy
BlackMould
BloodHound
Bonadan
BoomBox
BoxCaon
Brave Prince
Briba
Brute Ratel C4
Bumblebee
Bundlore
CALENDAR
CARROTBALL
CARROTBAT
CCBkdr
CHOPSTICK
COATHANGER
CORALDECK
CORESHELL
CSPY Downloader
Cachedump
CaddyWiper
Cadelspy
Calisto
CallMe
Cannon
Carbanak
Carberp
Carbon
Cardinal RAT
Catchamas
Caterpillar WebShell
ChChes
Chaes
Chaos
CharmPower
Cheerscrypt
Cherry Picker
China Chopper
Chinoxy
Chrommme
Clambling
Clop
Cloud

KeyboardInterrupt: 

In [ ]:
# Create a dictionary with variable names as keys and empty lists as initial values
matches_dict = {name: [] for name, _ in regex_list}

# Iterate through CSV content
with open('allsources_3x200k.csv', 'r') as file:
    csv_reader = csv.DictReader(file)
    
    for row in csv_reader:
        content = row['content']

        # Match regular expressions
        for name, regex in regex_list:
            print(name)
            if re.search(regex, content):
                matches_dict[name].append(row['source'])

# Print the dictionary with regex matches
print(matches_dict)

In [ ]:
# Create a dictionary to store counts for each regex pattern
regex_counts = defaultdict(int)
for key, value in matches_dict.items():
    regex_counts[key] = len(value)

# Print the dictionary with regex counts
print(dict(regex_counts))